# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import time
import gmaps
import os

# Import API key
from api_key import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
# Load file 
weather_data = pd.read_csv("../WeatherPy/output_data/cities.csv")
weather_data

,City_ID,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
0,0,ushuaia,75,AR,1610144381,41,-54.8000,-68.3000,57.20,18.41
1,1,margate,99,GB,1610144297,90,51.3813,1.3862,37.99,4.00
2,2,port alfred,47,ZA,1610144197,92,-33.5906,26.8910,70.00,1.01
3,3,dekar,100,BW,1610144391,74,-21.5333,21.9333,71.78,5.86
4,4,paamiut,98,GL,1610144394,80,61.9940,-49.6678,27.14,9.91
...,...,...,...,...,...,...,...,...,...,...
545,545,dobrinka,98,RU,1610145895,99,52.1653,40.4731,31.69,9.19
546,546,monkey bay,100,MW,1610145897,75,-14.0728,34.9201,79.27,8.84
547,547,qaqortoq,75,GL,1610145900,93,60.7167,-46.0333,32.00,2.30
548,548,filadelfia,90,US,1610145606,60,39.9523,-75.1638,37.00,4.54


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
# Configure gmaps
gmaps.configure(api_key=g_key)

# Store latitude and longitude in locations
locations = weather_data[["Lat", "Lng"]]

# Fill Nan values and covert to float 
humidity = weather_data["Humidity"]

In [4]:
# Plot heatmap
fig = gmaps.figure(center=(46.0, -5.0), zoom_level=2)

# Create heat layer
heat_layer = gmaps.heatmap_layer(locations, weights = humidity, dissipating=False, max_intensity=100, point_radius=3)

# Add layer
fig.add_layer(heat_layer)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [5]:
# Narrow down the DataFrame to find the ideal weather condition.

# A max temperature lower than 75 degrees but higher than 70.
ideal_weather_df = weather_data.loc[(weather_data["Max Temp"] < 75) & (weather_data["Max Temp"] > 70)]

# Wind speed less than 10 mph.
ideal_weather_df = ideal_weather_df.loc[(weather_data["Wind Speed"] < 10)]

# Zero cloudiness.
ideal_weather_df = ideal_weather_df.loc[(weather_data["Cloudiness"] == 0)]
                                         
# Drop any rows that don't contain all three conditions. 
ideal_weather_df = ideal_weather_df.dropna()
ideal_weather_df

,City_ID,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
17,17,kaduqli,0,SD,1610144427,29,11.0167,29.7167,73.94,8.88
59,59,adrar,0,MR,1610144550,25,20.5022,-10.0711,74.12,7.11
172,172,bilma,0,NE,1610144866,20,18.6853,12.9164,74.14,6.96
197,197,mecca,0,SA,1610144735,61,21.4267,39.8261,74.39,2.62
223,223,saint-joseph,0,RE,1610145005,68,-21.3667,55.6167,73.40,3.44
240,240,arlit,0,NE,1610145051,19,18.7369,7.3853,74.62,6.76
255,255,araouane,0,ML,1610145092,19,18.9048,-3.5265,71.71,4.18
273,273,umm lajj,0,SA,1610145141,53,25.0213,37.2685,74.71,6.15
329,329,koulikoro,0,ML,1610145294,19,14.0000,-7.7500,73.74,5.10
505,505,saint-pierre,0,RE,1610145788,73,-21.3393,55.4781,73.40,4.61


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [6]:
# Create a hotel_df
hotel_df = ideal_weather_df.loc[:,["City","Country", "Lat", "Lng"]]

# Add a "Hotel Name" column to the DataFrame.
hotel_df["Hotel Name"] = ""

# Display the result
hotel_df

,City,Country,Lat,Lng,Hotel Name
17,kaduqli,SD,11.0167,29.7167,
59,adrar,MR,20.5022,-10.0711,
172,bilma,NE,18.6853,12.9164,
197,mecca,SA,21.4267,39.8261,
223,saint-joseph,RE,-21.3667,55.6167,
240,arlit,NE,18.7369,7.3853,
255,araouane,ML,18.9048,-3.5265,
273,umm lajj,SA,25.0213,37.2685,
329,koulikoro,ML,14.0000,-7.7500,
505,saint-pierre,RE,-21.3393,55.4781,


In [7]:
# Create a params dictionary to update each iteration
params = {"type" : "lodging",
          "radius" : 5000,
          "key" : g_key}

# Loop through hotel_df and run a lat/long search for each city
for index, row in hotel_df.iterrows():
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    
    lat = row["Lat"]
    lng = row["Lng"]
    city_name = row["City"]

    # Update location key value 
    params["location"] = f"{lat},{lng}"
    
    # Assemble url and make request
    print(f"Retrieving Results for Index {index}: {city_name}.")
    response = requests.get(base_url, params=params).json()
    
    # Extract results
    results = response['results']
    
    # Save the hotel name to hotel_df
    try:
        print(f"Closest hotel in {city_name} is {results[0]['name']}.")
        hotel_df.loc[index, "Hotel Name"] = results[0]['name']

    # If there is no hotel available, show missing field
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")
        
    print("------------")
    
    # Delay the code to avoid an error
    time.sleep(1)

# Print end of search once searching is completed
print("End of Search")

Retrieving Results for Index 17: kaduqli.
Closest hotel in kaduqli is فندق الله جابو.
------------
Retrieving Results for Index 59: adrar.
Missing field/result... skipping.
------------
Retrieving Results for Index 172: bilma.
Closest hotel in bilma is Expeditions Ténére voyages.
------------
Retrieving Results for Index 197: mecca.
Closest hotel in mecca is Makkah Clock Royal Tower, A Fairmont Hotel.
------------
Retrieving Results for Index 223: saint-joseph.
Closest hotel in saint-joseph is "Plantation Bed and Breakfast.
------------
Retrieving Results for Index 240: arlit.
Closest hotel in arlit is Case de Passage De la Prefecture.
------------
Retrieving Results for Index 255: araouane.
Missing field/result... skipping.
------------
Retrieving Results for Index 273: umm lajj.
Closest hotel in umm lajj is Moon light Furnished Units.
------------
Retrieving Results for Index 329: koulikoro.
Missing field/result... skipping.
------------
Retrieving Results for Index 505: saint-pierre

In [8]:
# Display hotel_df
hotel_df

,City,Country,Lat,Lng,Hotel Name
17,kaduqli,SD,11.0167,29.7167,فندق الله جابو
59,adrar,MR,20.5022,-10.0711,
172,bilma,NE,18.6853,12.9164,Expeditions Ténére voyages
197,mecca,SA,21.4267,39.8261,"Makkah Clock Royal Tower, A Fairmont Hotel"
223,saint-joseph,RE,-21.3667,55.6167,"""Plantation Bed and Breakfast"
240,arlit,NE,18.7369,7.3853,Case de Passage De la Prefecture
255,araouane,ML,18.9048,-3.5265,
273,umm lajj,SA,25.0213,37.2685,Moon light Furnished Units
329,koulikoro,ML,14.0000,-7.7500,
505,saint-pierre,RE,-21.3393,55.4781,Lindsey Hôtel


In [9]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [10]:
# Add marker layer ontop of heat map
markers = gmaps.marker_layer(locations, info_box_content = hotel_info)
fig.add_layer(markers)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))